In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# @vla: custom parse options and quantize function
from lavis.args_parser import *
from lavis.quantize import quantize

import lavis.tasks as tasks
from lavis.common.config import Config

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
parser = args_parser()

In [4]:
CLI_INPUT = f'''
                 --cfg-path ret_flickr_eval.yaml
                 --visual-encoder-block-indices {','.join([str(i) for i in range(39)])}
                 --visual-encoder-block-modules qkv,proj,fc1,fc2
                 --visual-encoder-block-weight-bits 8
                 --qformer-layer-indices {','.join([str(i) for i in range(12)])}
                 --qformer-self-attention-modules query,key,value,dense
                 --qformer-self-attention-weight-bits 8
                 --qformer-cross-attention-modules query,key,value,dense
                 --qformer-cross-attention-weight-bits 6
                 --qformer-text-ff-modules intermediate,output
                 --qformer-text-ff-weight-bits 8
                 --qformer-img-ff-modules intermediate_query,output_query
                 --qformer-img-ff-weight-bits 4
                 --qformer-cls-modules transform,decoder
                 --qformer-cls-transform-weight-bits 3
                 --qformer-cls-decoder-weight-bits 1
                 --output-modules vision_proj,text_proj,itm_head
                 --vision-proj-weight-bits 8
                 --text-proj-weight-bits 7
                 --itm-head-weight-bits 6
             '''
             
             
# CLI_INPUT = f'''--cfg-path ret_flickr_eval.yaml --visual-encoder-block-modules qkv,proj,fc1,fc2'''

#  --visual-encoder-block-indices 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38 --visual-encoder-block-weight-bits 8'''

In [5]:
args = parser.parse_args(CLI_INPUT.split())
vars(args)

{'cfg_path': 'ret_flickr_eval.yaml',
 'options': None,
 'visual_encoder_block_modules': ['qkv', 'proj', 'fc1', 'fc2'],
 'visual_encoder_block_indices': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38],
 'visual_encoder_block_weight_bits': 8,
 'qformer_layer_indices': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
 'qformer_self_attention_modules': ['query', 'key', 'value', 'dense'],
 'qformer_self_attention_weight_bits': 8,
 'qformer_cross_attention_modules': ['query', 'key', 'value', 'dense'],
 'qformer_cross_attention_weight_bits': 6,
 'qformer_text_ff_modules': ['intermediate', 'output'],
 'qformer_text_ff_weight_bits': 8,
 'qformer_img_ff_modules': ['intermediate_query', 'output_query'],
 'qformer_img_ff_weight_bits': 4,
 'qformer_cls_modules': ['transform', 'decoder'],
 'qformer_cls_transform_weight_bits': 3,
 'qform

In [6]:
cfg = Config(args)
cfg

In [7]:
task = tasks.setup_task(cfg)
task

In [8]:
model = task.build_model(cfg)
model

MODEL_NAME: eva_clip_g
Position interpolate from 16x16 to 26x26


Blip2Qformer(
  (visual_encoder): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (blocks): ModuleList(
      (0-38): 39 x Block(
        (norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=1408, out_features=4224, bias=False)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=1408, out_features=1408, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=1408, out_features=6144, bias=True)
          (act): GELU(approximate='none')
          (fc2): Linear(in_features=6144, out_features=1408, bias=True)
          (drop): Dropout(p=0.0, inplace=False)
        )
      )
    )


In [9]:
quantize(model, args)

In [10]:
model

Blip2Qformer(
  (visual_encoder): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (blocks): ModuleList(
      (0-38): 39 x Block(
        (norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): NBitLinearDynamic(in_features=1408, out_features=4224, bias=False | w=8, a=32)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): NBitLinearDynamic(in_features=1408, out_features=1408, bias=True | w=8, a=32)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): NBitLinearDynamic(in_features=1408, out_features=6144, bias=True | w=8, a=32)
          (act): GELU(approximate='none')
          (fc2): NBitLinearDynamic(in_features=6144, out_features=1408, bia

In [13]:
for name, param in model.named_parameters():
    print(name)

query_tokens
temp
visual_encoder.cls_token
visual_encoder.pos_embed
visual_encoder.patch_embed.proj.weight
visual_encoder.patch_embed.proj.bias
visual_encoder.blocks.0.norm1.weight
visual_encoder.blocks.0.norm1.bias
visual_encoder.blocks.0.attn.q_bias
visual_encoder.blocks.0.attn.v_bias
visual_encoder.blocks.0.attn.qkv.weight
visual_encoder.blocks.0.attn.proj.weight
visual_encoder.blocks.0.attn.proj.bias
visual_encoder.blocks.0.norm2.weight
visual_encoder.blocks.0.norm2.bias
visual_encoder.blocks.0.mlp.fc1.weight
visual_encoder.blocks.0.mlp.fc1.bias
visual_encoder.blocks.0.mlp.fc2.weight
visual_encoder.blocks.0.mlp.fc2.bias
visual_encoder.blocks.1.norm1.weight
visual_encoder.blocks.1.norm1.bias
visual_encoder.blocks.1.attn.q_bias
visual_encoder.blocks.1.attn.v_bias
visual_encoder.blocks.1.attn.qkv.weight
visual_encoder.blocks.1.attn.proj.weight
visual_encoder.blocks.1.attn.proj.bias
visual_encoder.blocks.1.norm2.weight
visual_encoder.blocks.1.norm2.bias
visual_encoder.blocks.1.mlp.fc1.

TypeError: object of type 'generator' has no len()